## Основы глубокого обучения в NLP. CharRNN

План на сегодня: RNN - генератор имён

1. Базовая работа с текстом: токенизация, кодирование и декодирование
2. Рекуррентные архитектуры: RNN, LSTM, GRU
3. Обучение генерации == next token prediction

### 1. Готовим данные

In [1]:
# ! wget https://download.pytorch.org/tutorial/data.zip
# ! unzip data.zip

In [2]:
! ls data/names

Arabic.txt     English.txt    Irish.txt      Polish.txt     Spanish.txt
Chinese.txt    French.txt     Italian.txt    Portuguese.txt Vietnamese.txt
Czech.txt      German.txt     Japanese.txt   Russian.txt
Dutch.txt      Greek.txt      Korean.txt     Scottish.txt


In [3]:
! head -n 5 data/names/Chinese.txt

Ang
Au-Yong
Bai
Ban
Bao


#### 1.1. Пишем датасет и компоновщик батчей

Нам нужно:
1. Прочитать все имена из текстовых файлов
2. Закодировать каждое имя как последовательность целых чисел, предварительно добавив к именам символы начала и окончания (зачем?)
3. Сохранить пары (список токенов, id языка)
4. Сделать разбиение на train/test
5. Реализовать сборку примеров в батчи

In [4]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch
from torch import Tensor, nn
import torch.nn.functional as F

In [5]:
class NamesDataset(Dataset):
    # псевдоним для пары имя-язык
    _ItemPair = tuple[list[int], int]

    vocabulary: dict[str, int]
    languages: dict[int, str]
    names: list[_ItemPair]

    def __init__(self, datadir: Path) -> None:
        pad_token = ''
        bos_token = '?'  # beginning of sequence
        eos_token = '\n'  # end of sequence
        self.vocabulary = {pad_token: 0, bos_token: 1, eos_token: 2}
        self.special_tokens = {0, 1, 2}
        self.languages = {}
        self.names = []
        # iterate over files, update vocabulary, save name + language pairs
        for i, language_file in enumerate(datadir.glob("*.txt")):
            self.languages[i] = language_file.stem
            names = language_file.read_text().split('\n')
            for name in names:
                for letter in name:
                    # update vocab
                    if letter not in self.vocabulary:
                        self.vocabulary[letter] = len(self.vocabulary)
                # name -> list[int]
                encoded = self.encode(bos_token + name + eos_token)
                self.names.append((encoded, i))

        self._inverse_vocab = {value: key for key, value in self.vocabulary.items()}

    @property
    def vocab_size(self) -> int:
        return len(self.vocabulary)
    
    @property
    def num_classes(self) -> int:
        return len(self.languages)

    def encode(self, name: str) -> list[int]:
        return [self.vocabulary[letter] for letter in name]
    
    def decode(self, encoded: list[int], remove_special_tokens: bool = False) -> str:
        return ''.join(
            [
                self._inverse_vocab[idx]
                for idx in encoded
                if (idx not in self.special_tokens) or not remove_special_tokens
            ]
        )

    def __getitem__(self, index: int) -> tuple[list[int], int]:
        return self.names[index]
    
    def __len__(self) -> int:
        return len(self.names)

Проверка:

In [6]:
dataset = NamesDataset(Path("data/names/"))
tokens, label = dataset[4444]
print(tokens, label)
print(dataset.decode(tokens, remove_special_tokens=True), dataset.languages[label])


[1, 26, 17, 24, 2] 4
Chu Chinese


Разбивка датасета на трейн и тест:

In [7]:
from torch.utils.data.dataset import random_split
train_dataset, test_dataset = random_split(dataset, lengths=[0.9, 0.1])
print("Train size: ", len(train_dataset))
print("Test size: ", len(test_dataset))

Train size:  18083
Test size:  2009


Упаковка в батчи:

In [8]:
def collate_fn(batch: list[tuple[list[int], int]]) -> tuple[Tensor, Tensor]:
    encoded, lang_ids = zip(*batch)
    max_len = max(map(len, encoded))
    x = torch.zeros((len(encoded), max_len), dtype=int)
    for i, seq in enumerate(encoded):
        x[i, :len(seq)] = torch.tensor(seq)
    
    return x, torch.tensor(list(lang_ids))

In [9]:
batch = [train_dataset[i] for i in range(8)]
tokens, labels = collate_fn(batch)
print("Tokens shape: ", tokens.shape, "\nLabels shape: ", labels.shape)

Tokens shape:  torch.Size([8, 14]) 
Labels shape:  torch.Size([8])


Собираем в загрузчик данных:

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
tokens, labels = next(iter(train_loader))
print("Tokens shape: ", tokens.shape, "\nLabels shape: ", labels.shape)

Tokens shape:  torch.Size([32, 14]) 
Labels shape:  torch.Size([32])


### 2. Пишем простую RNN

Начнём с написания RNNCell - одного рекуррентного блока

<img src="https://i.stack.imgur.com/02KvP.png" style="background:white" width="600"/>

<!-- <img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" style="background:white" width="600"/> -->



In [11]:
class RNNCell(nn.Module):
    """
    (x_{t}, h_{t-1}) -> h_{t}
    """
    def __init__(self, input_dim: int, hidden_dim: int) -> None:
        super().__init__()
        self.linear = nn.Linear(input_dim+hidden_dim, hidden_dim)

    def forward(self, x: Tensor, h: Tensor) -> Tensor:
        # x: B x input_dim
        # h: B x hidden_dim
        h = torch.cat([x, h], dim=1)
        h = self.linear(h)
        return F.tanh(h)


Проверка:

In [12]:
batch_size = 4
input_dim = 10
hidden_dim = 8
cell = RNNCell(input_dim, hidden_dim)
h = torch.zeros(1, hidden_dim)
# расширяем до размеров батча
h_expanded = h.expand((batch_size, -1))
x = torch.randn(batch_size, input_dim)
h_new = cell.forward(x, h_expanded)
print(h_new.shape)

torch.Size([4, 8])


**Упражнение 1**: реализуйте более сложно устроенную LSTMCell, где теперь есть:
1. два внутренних состояния: cell state $c_t$ и hidden state $h_t$
2. набор гейтов для управления обновлениями состояний

[blog post](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
<!-- <img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" style="background:white" width="600"/> -->

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png" style="background:white" width="500"/>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png" style="background:white" width="500"/>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png" style="background:white" width="500"/>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" style="background:white" width="500"/>

**Упражнение 2**: реализуйте GRUCell

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png" style="background:white" width="500"/>

Опишем модель, состоящую из следующих блоков:
1. `embed`: кодирует входные токены в векторы размера `hidden_dim`
2. `rnn`: наша рекуррентная ячейка
3. `output`: восстанавливает логиты из скрытого состояния `h`

In [13]:

class RNN(nn.Module):
    def __init__(self, vocab_size: int, hidden_dim: int) -> None:
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_dim)
        self.init_h = nn.Parameter(data=torch.randn(1, hidden_dim))
        self.rnn = RNNCell(hidden_dim, hidden_dim)
        self.lm_head = nn.Linear(hidden_dim, vocab_size)


    def forward(self, x: Tensor) -> Tensor:
        # x: B x T
        # embed(x): B x T -> B x T x hidden_dim
        B, T = x.shape

        x = self.embed(x)  # B x T x hidden_dim
        h = self.init_h.expand((B, -1)) # B x hidden_dim

        logits = [] # T x B x V
        for t in range(T):
            xt = x[:, t, :]
            h = self.rnn.forward(xt, h)  # B x hidden
            y = self.lm_head(h).unsqueeze(1)  # B x 1 x hidden
            logits.append(y)
            # save prediction for step t + 1

        # lm_head: B x T x hidden -> B x T x V
        return torch.cat(logits, dim=1)



**Упражнение 3**. Добавьте в класс `RNN` возможность
   1. Нескольких последовательных рекуррентных слоёв
   2. Выбора другого типа рекуррентной ячейки (`GRU`, `LSTM`)

Проверка:

In [14]:
hidden_dim = 32
model = RNN(
    vocab_size=dataset.vocab_size,
    hidden_dim=hidden_dim,
)
model.forward(tokens).shape

torch.Size([32, 14, 90])

### 3. Функция для генерации

Схема:
1. Подаём на вход произвольный префикс имени, можно только токен начала
2. Проходимся моделью по префиксу, получаем логиты для следующего токена
3. Семплируем новый токен, добавляем его к префиксу, возвращаемся к шагу 1.
4. Критерии остановки:
   - встретили символ окончания строки
   - сгенерировали максимальное число новых токенов

In [15]:
@torch.no_grad()
def generate(model: RNN, idx: Tensor, max_new_tokens: int) -> Tensor:
    # idx: B x T
    for t in range(max_new_tokens):
        logits = model.forward(idx)[:, -1]  # B x T x V
        probs = F.softmax(logits, dim=1)  # B x V
        new_token = torch.multinomial(probs, 1)
        idx = torch.cat([idx, new_token], dim=1)

    return idx


**Упражнение 4**. Модифицируйте функцию `generate`, чтобы она при семплировании учитывала
   - $k$ наиболее вероятных токенов (параметр `top_k: int`)
   - только токены, дающие в сумме вероятность не меньше $p$ (параметр `top_p: int`)
   - температуру для `softmax`:


      $\begin{aligned}\text{softmax}(x_i, \tau) = \frac{\exp(x_i / \tau)}{\sum_j \exp(x_i / \tau)} \end{aligned}$

Ещё понадобится функция, которая умеет декодировать выход из функции `generate`

In [16]:
def batch_decode(out_tokens: Tensor) -> list[str]:
    decoded_strings = []
    for x in out_tokens:
        decoded_strings.append(dataset.decode(x.tolist(), remove_special_tokens=True))

    return decoded_strings
        

Сгенерируем несколько "имён" для проверки, начиная со $\texttt{<BOS>}$ токена:

In [17]:
samples = generate(model, idx=torch.full(size=(4, 1), fill_value=1, dtype=int), max_new_tokens=40)
print('\n'.join(batch_decode(samples)))

üàżDpßgŚ,ñ'ÉkŚsúöuzzçòOmDá-êW'õwYçùG1
ąążòjñYjXłvąêÉXEöAPúçzRółbX'ßóObetUùä
Hń1êLpgCÁ1Vòvgł,-èiŚkąuZF-yUõhłŚAyagkŻAD
RNúzéJAQYńùiIklhÉs:xlWrÉżüéñMuVVRKVñéçPS


### 4. Цикл обучения

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [19]:
logits = model.forward(tokens)
print(logits.shape)
print(tokens.shape)

torch.Size([32, 14, 90])
torch.Size([32, 14])


In [20]:
F.cross_entropy(
    logits[:, :-1].reshape(-1, dataset.vocab_size),
    tokens[:, 1:].reshape(-1),
)

tensor(4.4815, grad_fn=<NllLossBackward0>)

In [21]:
for epoch in range(5):
    for tokens, _ in train_loader:
        logits = model.forward(tokens)  # B x T x V
        loss = F.cross_entropy(
            logits[:, :-1].reshape(-1, dataset.vocab_size),
            tokens[:, 1:].reshape(-1),
        )
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss)


tensor(1.7738, grad_fn=<NllLossBackward0>)
tensor(1.8683, grad_fn=<NllLossBackward0>)
tensor(1.9535, grad_fn=<NllLossBackward0>)
tensor(1.9571, grad_fn=<NllLossBackward0>)
tensor(1.9983, grad_fn=<NllLossBackward0>)


In [22]:
samples = generate(model, idx=torch.full(size=(20, 1), fill_value=1, dtype=int), max_new_tokens=40)
print('\n'.join(batch_decode(samples)))

Zheson
Vuro
Tapi
Archeev
Admochiy
Aialev
Tutsaneno
Jepeva
Horondon
Vayurov
Dekfleng
Zhandull
Bamabnichkov
Lanhnov
Timo
Javy
Ribinzgi
Babapany
Vallin
Zhelyur


### Упражнения

5. Модифицируйте вычисление ошибки, чтобы не считать её для токенов, отвечающих за паддинг. Повлияло ли это на скорость обучения модели?
6. Добавьте в генерацию входное условие: язык для генерируемого имени
7. Используйте `nn.LSTM` и `nn.GRU` вместо самописных моделей, сравните результаты. 
8. Реализуйте модель для классификации имён по языкам